In [1]:
! pip install datasets langchain_community sentence_transformers chromadb langchain_openai ragas==0.0.19 --q
! pip install transformers[torch] --q
! pip install accelerate==0.27.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
cudf 24.6.1 requires cuda-python<12.0a0,>=11.7.1, but you have cu

In [2]:
from tqdm import tqdm

In [3]:
import os
import pandas as pd
from datasets import Dataset
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("OPENAI_KEY")
os.environ["WANDB_API_KEY"] = user_secrets.get_secret("wandb_api")

In [4]:
from datasets import load_dataset
 
# Load dataset from the hub
train_dataset = load_dataset("neural-bridge/rag-dataset-12000", split="train")
train_dataset = train_dataset.rename_column("question", "anchor")
train_dataset = train_dataset.rename_column("context", "positive")
train_dataset = train_dataset.remove_columns('answer')

df = train_dataset.to_pandas()
print(df.isnull().sum())
df.dropna(inplace=True)
print(df.isnull().sum())
train_dataset  = Dataset.from_pandas(df)

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2400 [00:00<?, ? examples/s]

positive    0
anchor      2
dtype: int64
positive    0
anchor      0
dtype: int64


In [5]:
train_dataset = train_dataset.remove_columns('__index_level_0__')
train_dataset

Dataset({
    features: ['positive', 'anchor'],
    num_rows: 9598
})

In [6]:
test = pd.read_csv('/kaggle/input/ragdataset/ragtestset.csv')
test = test.loc[:,['context','question']].rename(columns={'context': 'positive', 'question': 'anchor'})
eval_dataset  = Dataset.from_pandas(test)

In [7]:
print(train_dataset) , print(eval_dataset)

Dataset({
    features: ['positive', 'anchor'],
    num_rows: 9598
})
Dataset({
    features: ['positive', 'anchor'],
    num_rows: 500
})


(None, None)

In [8]:
from sentence_transformers import SentenceTransformer
import torch

model_name = "BAAI/bge-base-en-v1.5"
model = SentenceTransformer(model_name)

2024-07-25 04:54:38.750491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 04:54:38.750604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 04:54:38.879433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers

In [10]:
# 3. Define a loss function
loss = MultipleNegativesRankingLoss(model)

In [11]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/bge-base-en-v1.5-ft_ragds",
    # Optional training parameters:
    num_train_epochs=30,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    run_name="bge-base-en-v1.5-finetuned_ragds_v1",  # Will be used in W&B if `wandb` is installed
)

In [12]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss
)

In [13]:
trainer.train()

wandb: Currently logged in as: aritra-slg. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240725_045459-17677c2n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bge-base-en-v1.5-finetuned_ragds_v1
wandb: ⭐️ View project at https://wandb.ai/aritra-slg/sentence-transformers
wandb: 🚀 View run at https://wandb.ai/aritra-slg/sentence-transformers/runs/17677c2n


Step,Training Loss,Validation Loss
100,0.132500,0.055011
200,0.080600,0.029731
300,0.060500,0.019043
400,0.060900,0.015842
500,0.037100,0.013695
600,0.034900,0.014407
700,0.050700,0.012663
800,0.019700,0.011971
900,0.041200,0.014516
1000,0.033000,0.012199


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=28800, training_loss=0.002726704316197078, metrics={'train_runtime': 20445.7298, 'train_samples_per_second': 14.083, 'train_steps_per_second': 1.409, 'total_flos': 0.0, 'train_loss': 0.002726704316197078, 'epoch': 30.0})

In [14]:
# 8. Save the trained model
model.save_pretrained("models/bge-base-en-v1.5-finetuned_ragds_v1")

# 9. (Optional) Push it to the Hugging Face Hub
model.push_to_hub("bge-base-en-v1.5-finetuned_ragds_v1")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/aritrasen/bge-base-en-v1.5-finetuned_ragds_v1/commit/ba4d92ef129d540f384e84b2c587ab1a4daa3204'